What I want initially is to use the whole 3000 stocks.
However due to the capacity of my laptop, now, I can only use one of them to form a baby-version model.

In [74]:
import numpy as np
import pandas as pd
import os
import copy

I use the most resent 10 years' data to build my model.
And split the train and test set by proportion of 7:3, which means data from 2006-09-01 to 2013-09-02 is the training set, while data from 2013-09-02 to 2016-09-01 is the test set.

In [75]:
database={}
trainset={}
testset={}
#for i in range(4000):
    
    #import my data
    
#    name=str('060')+str(i).zfill(4)
#    if os.path.isfile('e:/data/'+str(name)+'.csv'):
name='0600000'
data=pd.read_csv('e:/data/'+str(name)+'.csv',encoding='gb2312')
data.columns=['date','code','name','clsPrc','highPrc','lowPrc','openPrc','clsPrc_lastday','diff',
              'return', 'turnover_rate','volumn','turnover_value','total_market_value','current_market_value','hh']
data=data.drop(['hh'],axis=1)

#select the data

if sum(data['date']=='2006-09-01')==1 and sum(data['date']=='2013-09-02')==1:

    #add some features

    for lag in [1,5]:     ## [1,2,3,4,5,6,7,8,9,10] will be better
        for col_name in ['clsPrc','highPrc','lowPrc','openPrc','clsPrc_lastday','diff',
              'return', 'turnover_rate','volumn','turnover_value','total_market_value','current_market_value']:
            data[str(col_name)+'lag'+str(lag)]=data[str(col_name)].shift(-lag)


    #split the training set and the test set

    cutoff1=(data[data['date']=='2006-09-01'].index)
    cutoff2=(data[data['date']=='2013-09-02'].index)
    data=data[data.index<cutoff1[0]]
    train=data[data.index>cutoff2[0]]
    test =data[data.index<cutoff2[0]]
    database.update({str(name):data})
    trainset.update({'train_'+str(name):train})
    testset.update({'test_'+str(name):test})

In [83]:
train=trainset['train_0600000']
test=testset['test_0600000']

for i in ['clsPrclag1','highPrclag1','lowPrclag1','openPrclag1','clsPrc_lastdaylag1','difflag1','returnlag1',
               'turnover_ratelag1','volumnlag1','turnover_valuelag1','total_market_valuelag1','current_market_valuelag1',
               'clsPrclag5','highPrclag5','lowPrclag5','openPrclag5','clsPrc_lastdaylag5','difflag5', 'returnlag5', 'turnover_ratelag5',
               'volumnlag5','turnover_valuelag5','total_market_valuelag5','current_market_valuelag5']:
    
    train[str(i)]=pd.to_numeric(train[str(i)].values,errors='coerce')
    where_are_NaNs = np.isnan(train[str(i)])
    train[str(i)][where_are_NaNs] = 0
    
    test[str(i)]=pd.to_numeric(test[str(i)].values,errors='coerce')
    where_are_NaNs = np.isnan(test[str(i)])
    test[str(i)][where_are_NaNs] = 0
    
X_train=train[['clsPrclag1','highPrclag1','lowPrclag1','openPrclag1','clsPrc_lastdaylag1','difflag1','returnlag1',
               'turnover_ratelag1','volumnlag1','turnover_valuelag1','total_market_valuelag1','current_market_valuelag1',
               'clsPrclag5','highPrclag5','lowPrclag5','openPrclag5','clsPrc_lastdaylag5','difflag5', 'returnlag5', 'turnover_ratelag5',
               'volumnlag5','turnover_valuelag5','total_market_valuelag5','current_market_valuelag5']]
y_train=pd.to_numeric(train['diff'].values,errors='coerce')

X_test=test[['clsPrclag1','highPrclag1','lowPrclag1','openPrclag1','clsPrc_lastdaylag1','difflag1', 'returnlag1', 
            'turnover_ratelag1','volumnlag1','turnover_valuelag1','total_market_valuelag1','current_market_valuelag1',
            'clsPrclag5','highPrclag5','lowPrclag5','openPrclag5','clsPrc_lastdaylag5','difflag5', 'returnlag5', 'turnover_ratelag5',
            'volumnlag5','turnover_valuelag5','total_market_valuelag5','current_market_valuelag5']]
y_test=pd.to_numeric(test['diff'].values,errors='coerce')

where_are_NaNs1 = np.isnan(y_train)
y_train[where_are_NaNs1] = 0
where_are_NaNs2 = np.isnan(y_test)
y_test[where_are_NaNs2] = 0
y_train=(y_train>0)*1
y_test=(y_test>0)*1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sel

In [81]:
type(y_test[0])

numpy.float64

In [85]:
# 1. import
from sklearn.neural_network import MLPClassifier

# 2. instantiate
mlp = MLPClassifier()

# 3. train
mlp.fit(X_train, y_train)

# 4. predict
p = mlp.predict(X_test)

# 5. evaluate
from sklearn import metrics
metrics.accuracy_score(p, y_test)

0.50068212824010916